<a href="https://colab.research.google.com/github/MadhuReddy001/Image-Captioning/blob/main/Spam_filter_for_Quora_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
!wget https://www.dropbox.com/scl/fo/6cbk5ktsjhcd9kqxn01qj/AGuGEKF2SeUhJwGC4ju98vU?rlkey=25axzya0eum5h4qv91hpiies4&e=1&dl=0

--2025-01-02 16:58:52--  https://www.dropbox.com/scl/fo/6cbk5ktsjhcd9kqxn01qj/AGuGEKF2SeUhJwGC4ju98vU?rlkey=25axzya0eum5h4qv91hpiies4
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uce83b6f682e8276d7f1a9eed542.dl.dropboxusercontent.com/zip_download_get/CE0-uyhT5uWoO-FZ8W7CO-4yDtTG26baB8jP9yVFffYdEOGEWAJ2ODZ_U5kd0Teo2neeJjvArThEz_5gurORYObJWV840xxXM5sKDJESHIwcPQ# [following]
--2025-01-02 16:58:53--  https://uce83b6f682e8276d7f1a9eed542.dl.dropboxusercontent.com/zip_download_get/CE0-uyhT5uWoO-FZ8W7CO-4yDtTG26baB8jP9yVFffYdEOGEWAJ2ODZ_U5kd0Teo2neeJjvArThEz_5gurORYObJWV840xxXM5sKDJESHIwcPQ
Resolving uce83b6f682e8276d7f1a9eed542.dl.dropboxusercontent.com (uce83b6f682e8276d7f1a9eed542.dl.dropboxusercontent.com)... 162.125.5.15, 2620:100:601d:15::a27d:50f
Connecting to uce83b6f682e8276d7f1a9eed542.dl.dropbo

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize


In [ ]:
filter = pd.read_csv(r'/content/AGuGEKF2SeUhJwGC4ju98vU?rlkey=25axzya0eum5h4qv91hpiies4', sep=',', encoding='latin-1', header=1)

In [ ]:
filter

,BN,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0.0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0.0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0.0
...,...,...,...
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0.0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0.0
1306121,ffffed09fedb5088744a,Who wins in a battle between a Wolverine and a...,0.0
1306122,PKÐÞ¾±´>g´>gPK,NaN,NaN


In [ ]:
filter.isnull().sum()

,0
BN,0
question_text,2
target,2


In [ ]:
for col in ['question_text','target']:
  print(col,',',filter[col].value_counts())

question_text , question_text
How did Quebec nationalists see their province as a nation in the 1960s?                                      1
What is the best method to teach the pronunciation of English words to kids?                                  1
How much mark should we get to pass in NEET exam?                                                             1
Iâm 13 and overweight. Aside from having a healthy diet, what exercises can I do at home to lose weight?    1
How do you get perfect teeth without using braces?                                                            1
                                                                                                             ..
Why are all psychotherapists so bad at their jobs?                                                            1
How many babies are born each day in India?                                                                   1
What are the milk substitutes for baking a cake?                          

In [ ]:
filter.drop(index=[1306122, 1306123], inplace=True)

In [ ]:
filter.isnull().sum()

,0
BN,0
question_text,0
target,0


In [ ]:
filter.drop(columns=['BN'], inplace=True)

In [ ]:
filter.dtypes

,0
question_text,object
target,float64


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer # Changed import statement to use tensorflow.keras.preprocessing
from tensorflow.keras.preprocessing import sequence # Changed import statement to use tensorflow.keras.preprocessing
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model

from nltk import word_tokenize

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [ ]:
!wget https://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz

--2025-01-02 17:00:57--  https://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17329808 (17M) [application/x-gzip]
Saving to: ‘news20.tar.gz’

news20.tar.gz       100%[===================>]  16.53M  4.80MB/s    in 3.4s    

2025-01-02 17:01:01 (4.80 MB/s) - ‘news20.tar.gz’ saved [17329808/17329808]



In [ ]:
!tar -xvzf /content/news20.tar.gz

Streaming output truncated to the last 5000 lines.
20_newsgroup/soc.religion.christian/20607
20_newsgroup/soc.religion.christian/20608
20_newsgroup/soc.religion.christian/20609
20_newsgroup/soc.religion.christian/20579
20_newsgroup/soc.religion.christian/20580
20_newsgroup/soc.religion.christian/20581
20_newsgroup/soc.religion.christian/20582
20_newsgroup/soc.religion.christian/20583
20_newsgroup/soc.religion.christian/20584
20_newsgroup/soc.religion.christian/20585
20_newsgroup/soc.religion.christian/20586
20_newsgroup/soc.religion.christian/20587
20_newsgroup/soc.religion.christian/20588
20_newsgroup/soc.religion.christian/20589
20_newsgroup/soc.religion.christian/20590
20_newsgroup/soc.religion.christian/20591
20_newsgroup/soc.religion.christian/20592
20_newsgroup/soc.religion.christian/20593
20_newsgroup/soc.religion.christian/20594
20_newsgroup/soc.religion.christian/20595
20_newsgroup/soc.religion.christian/20596
20_newsgroup/soc.religion.christian/20597
20_newsgroup/soc.religion

In [ ]:

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
import os, sys
for name in sorted(os.listdir('/content/20_newsgroup')):
    path = os.path.join('/content/20_newsgroup', name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

Found 19997 texts.


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer # Changed import statement to use tensorflow.keras.preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences # Changed import statement to use tensorflow.keras.preprocessing


tokenizer = Tokenizer(nb_words=50000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=100)

# Assuming 'to_categorical' is imported from tensorflow.keras.utils
from tensorflow.keras.utils import to_categorical  # Import to_categorical
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(0.2 * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Found 174074 unique tokens.
Shape of data tensor: (19997, 100)
Shape of label tensor: (19997, 20)


In [ ]:
EMBEDDING_DIM = 100  # or any desired embedding dimension, e.g., 50, 200, 300
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=100,
                            trainable=False)

In [ ]:
X=filter['question_text']
Y=filter['target']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [ ]:
class_weight=class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
print(class_weight)

[0.5329878  8.07855917]


In [ ]:
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: 0.5329878007724749, 1: 8.078559168715499}

In [ ]:
x_train

,question_text
91925,How do you imagine the future of shopping in a...
1305387,"What are the hazards of copper(II) nitrate, tr..."
875559,Will you get an entry ban from the U.A.E if yo...
981960,Is it illegal to exclude legal residents from ...
721748,What speakers should I buy for huge dance part...
...,...
441884,What are some biotechnology companies that usu...
318805,What is the difference between twin flames and...
740133,Where can I get servo motor drivers in Delhi?
879486,What are some tips for round brushing and blow...


In [ ]:
from nltk import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

361


In [ ]:
np.percentile(sent_lens,99)

44.0

In [ ]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [ ]:
tk.word_index

{'the': 1,
 'what': 2,
 'is': 3,
 'a': 4,
 'to': 5,
 'in': 6,
 'of': 7,
 'i': 8,
 'how': 9,
 'and': 10,
 'do': 11,
 'are': 12,
 'for': 13,
 'you': 14,
 'can': 15,
 'why': 16,
 'it': 17,
 'my': 18,
 'that': 19,
 'if': 20,
 'with': 21,
 'or': 22,
 'on': 23,
 'have': 24,
 'be': 25,
 'does': 26,
 'from': 27,
 'your': 28,
 'an': 29,
 'which': 30,
 'should': 31,
 'when': 32,
 'get': 33,
 'best': 34,
 'would': 35,
 'as': 36,
 'people': 37,
 'some': 38,
 'there': 39,
 'who': 40,
 'will': 41,
 'like': 42,
 'at': 43,
 'not': 44,
 'about': 45,
 'by': 46,
 'they': 47,
 'did': 48,
 'was': 49,
 'any': 50,
 'we': 51,
 'so': 52,
 'good': 53,
 'me': 54,
 'their': 55,
 'one': 56,
 'india': 57,
 'has': 58,
 'after': 59,
 'most': 60,
 'where': 61,
 'make': 62,
 'this': 63,
 'but': 64,
 'more': 65,
 'all': 66,
 'think': 67,
 'many': 68,
 'between': 69,
 'time': 70,
 'than': 71,
 'much': 72,
 'other': 73,
 'life': 74,
 'someone': 75,
 'use': 76,
 'he': 77,
 'out': 78,
 'way': 79,
 'am': 80,
 'know': 81,
 'u

In [ ]:
vocab_size

196247

In [ ]:
seq_train_matrix.shape

(1044897, 44)

In [ ]:
seq_train

[[9, 11, 14, 3114, 1, 251, 7, 2539, 6, 4, 200, 2611, 79],
 [2, 12, 1, 11993, 7, 3226, 2118, 8332, 64433],
 [41,
  14,
  33,
  29,
  1552,
  1611,
  27,
  1,
  330,
  4,
  383,
  20,
  14,
  734,
  4,
  110,
  685,
  10,
  420,
  5,
  4808],
 [3, 17, 788, 5, 9977, 484, 5127, 27, 363, 1176, 6, 1, 92],
 [2, 2441, 31, 8, 183, 13, 1586, 2284, 2304, 8, 3017],
 [2, 12, 1, 34, 4354, 88124, 13, 37, 21, 16034],
 [9, 11, 8, 121, 4, 53549, 6, 600],
 [2, 3, 1, 257, 427, 13, 844, 1584],
 [9,
  68,
  3432,
  7,
  8017,
  286,
  1030,
  14596,
  134,
  15,
  25,
  750,
  27,
  4,
  8017,
  46674,
  14596,
  134],
 [26, 31458, 829, 19716, 242, 3395],
 [16,
  11,
  893,
  306,
  10,
  714,
  3095,
  4,
  9713,
  13053,
  5096,
  41933,
  10,
  26040,
  1370,
  321],
 [16, 3, 17, 725, 5, 41934, 214, 144, 1567],
 [9,
  68,
  635,
  31,
  8,
  184,
  607,
  5,
  1229,
  3342,
  211,
  20,
  8,
  80,
  11354,
  13,
  20690,
  600],
 [30, 562, 2605, 1196, 7973, 394],
 [20,
  56,
  99,
  15,
  1031,
  46,
  1

In [ ]:
x_train.shape

(1044897,)

In [ ]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [ ]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [ ]:
 len(embedding_matrix)

174075

In [ ]:
embedding_matrix.shape

(174075, 100)

In [ ]:
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
inputs = Input(name='text_input',shape=[max_len])
# vocab_size + 1 should match the first dimension of embedding_matrix, and
# 300 should be changed to 100 to match the second dimension.
embed=Embedding(len(embedding_matrix), 100, # Changed vocab_size+1 to len(embedding_matrix) and 300 to 100
                input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [ ]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)   │ (None, 44)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 44, 100)        │     17,407,500 │ text_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, 44)             │              0 │ text_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 512)            │      1,255,424 │ embedding_3[0][0],     │
│                           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 512)            │              0 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 256)            │        131,328 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 256)            │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 128)            │         32,896 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 25)             │          3,225 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 25)             │              0 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 1)              │             26 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 18,830,399 (71.83 MB)

 Trainable params: 1,422,899 (5.43 MB)

 Non-trainable params: 17,407,500 (66.40 MB)

In [ ]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.keras' # Changed .h5 to .keras
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,
                             save_best_only=True)

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
!pip install imbalanced-learn

In [ ]:
inputs = Input(name='text_input',shape=[max_len])
# Use vocab_size + 1 to match the size of tk.word_index, and
# EMBEDDING_DIM (100) for the embedding dimension.
embed = Embedding(vocab_size + 1, EMBEDDING_DIM,  # Changed to vocab_size + 1 and EMBEDDING_DIM
                input_length=max_len,mask_zero=True,
                # If you want to use pre-trained embeddings, make sure
                # they are compatible with your vocabulary size. Otherwise,
                # initialize the weights randomly.
                # weights=[embedding_matrix],  # Remove or adjust weights if necessary
                trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from imblearn.over_sampling import RandomOverSampler

# ... (your previous code) ...

# Oversample the minority class
ros = RandomOverSampler(random_state=42)  # Initialize RandomOverSampler
seq_train_matrix_resampled, y_train_resampled = ros.fit_resample(
    seq_train_matrix, y_train
)  # Resample the training data

# ... (your model definition) ...

# Compile the model after it's defined
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model using the resampled data
model.fit(
    seq_train_matrix_resampled,  # Use resampled data
    y_train_resampled,  # Use resampled labels
    epochs=15,
    batch_size=1000,
    validation_data=(seq_test_matrix, y_test),
    callbacks=[earlystop, checkpoint]
)  # Removed class_weight

# ... (rest of your code) ...

Epoch 1/15
 427/1961 ━━━━━━━━━━━━━━━━━━━━ 2:54:28 7s/step - accuracy: 0.7283 - loss: 0.5486

In [ ]:
# In cell 80, change the file path to match the one used for saving the model:
best_model = load_model('/content/weights-04-0.2007.keras') # Changed .h5 to .keras

In [ ]:
best_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)   │ (None, 44)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 44, 300)        │     58,923,600 │ text_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_1 (NotEqual)    │ (None, 44)             │              0 │ text_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 512)            │      1,665,024 │ embedding_1[0][0],     │
│                           │                        │                │ not_equal_1[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 512)            │              0 │ lstm_1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 256)            │        131,328 │ dropout_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 256)            │              0 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 128)            │         32,896 │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 25)             │          3,225 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 25)             │              0 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 1)              │             26 │ dropout_7[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 64,421,099 (245.75 MB)

 Trainable params: 1,832,499 (6.99 MB)

 Non-trainable params: 58,923,600 (224.78 MB)

 Optimizer params: 3,665,000 (13.98 MB)

In [ ]:
# Load the best model's weights
best_model_weights = load_model('/content/weights-04-0.2007.keras').get_weights()

# Recreate the model with use_cudnn=False
inputs = Input(name='text_input', shape=[max_len])
embed = Embedding(vocab_size + 1, 300, input_length=max_len, mask_zero=True,
                  weights=[embedding_matrix], trainable=False)(inputs)
lstm_layer1 = LSTM(512, use_cudnn=False)(embed)  # Set use_cudnn=False
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256, activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128, activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25, activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1, activation='sigmoid')(drop4)

recreated_model = Model(inputs=inputs, outputs=final_layer)

# Load the weights into the recreated model
recreated_model.set_weights(best_model_weights)

# Prediction with the recreated model
p = recreated_model.predict(seq_test_matrix)
roc_auc_score(y_test, p)

8164/8164 ━━━━━━━━━━━━━━━━━━━━ 43s 5ms/step


0.9644149735158491